In [1]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore') 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [62]:
apiKey = '1d49e941371e685a965fba06520cda68'

In [120]:
# 카카오 api 를 활용해서, 도로명 주소를 통해 -> 건물에 대한 상세 주소 알기
def address2api(DF,apiKey):
  cnt=0
  addressCopy = DF.copy()
  headers = {'Authorization': 'KakaoAK {}'.format(apiKey) }
  addressCopy['byAPI'] =None
  for idx in tqdm(range(len(addressCopy))):
    try:

      address = addressCopy.at[idx,'주소']

      searching = address
      url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(searching)
      data = requests.post(url,headers=headers)
      result = json.loads(data.text)
      documents = result['documents']
      addressCopy.at[idx,'byAPI'] = documents[0]
    except:
    # 값이 나오지 않은 주소 '충청남도 천안동남구 문화동 112-17 힐스테이트천안' 는 ->'충청남도 천안시 동남구 옛시청길 29' 도로 주소명으로 대체하여, 카카오 api 재호출을 통해서 값을 넣어줌
      '''
      searching = '충청남도 천안시 동남구 옛시청길 29'

      url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(searching)
      headers = {'Authorization': 'KakaoAK {}'.format(apiKey) }

      data = requests.post(url,headers=headers)
      result = json.loads(data.text)
      documents = result['documents']
      addressCopy.at[idx,'byAPI'] = documents[0]
      '''
      
      cnt+=1
      if cnt%50==0:
        print('쿼리값을 못찾는중:',cnt) 
      pass

  return addressCopy



In [187]:
def address2apiResidual(searching,apiKey) -> dict:
    headers = {'Authorization': 'KakaoAK {}'.format(apiKey) }
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(searching)
    data = requests.post(url,headers=headers)
    result = json.loads(data.text)
    documents = result['documents']
    return documents[0]

In [2]:
apt = pd.read_pickle('/Users/iseunghyeon/Downloads/addressByAPI_apt_0816.pickle')
all_building = pd.read_pickle('/Users/iseunghyeon/Downloads/addressByAPI_all_building_0816.pickle')
all_building.head()

,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부,byAPI,건물명
0,4413025629101030004000001,1,4413325629,충청남도,천안시 서북구,직산읍,모시리,0,103,4,1,{'address': {'address_name': '충남 천안시 서북구 직산읍 모...,NA
1,4413025632102760000000001,1,4413325632,충청남도,천안시 서북구,직산읍,신갈리,0,276,0,1,{'address': {'address_name': '충남 천안시 서북구 직산읍 군...,(주)두호스테인레스
2,4413034027103000000000001,1,4413134027,충청남도,천안시 동남구,성남면,신사리,0,300,0,1,{'address': {'address_name': '충남 천안시 동남구 성남면 대...,NA
3,4413034027103000000000001,3,4413134027,충청남도,천안시 동남구,성남면,신사리,0,300,3,0,{'address': {'address_name': '충남 천안시 동남구 성남면 신...,NA
4,4413034027103000000000001,5,4413134027,충청남도,천안시 동남구,성남면,신사리,0,300,5,0,{'address': {'address_name': '충남 천안시 동남구 성남면 신...,NA


In [3]:
villa = all_building[all_building['건물명'].str.contains('빌라')]
villa

,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부,byAPI,건물명
551,4413110100102140000000001,1,4413110100,충청남도,천안시 동남구,대흥동,NaN,0,214,0,1,{'address': {'address_name': '충남 천안시 동남구 대흥동 2...,빌라
607,4413110100102370000022867,1,4413110100,충청남도,천안시 동남구,대흥동,NaN,0,237,0,1,{'address': {'address_name': '충남 천안시 동남구 대흥동 2...,반석빌라
687,4413110200100010001011579,1,4413110200,충청남도,천안시 동남구,성황동,NaN,0,1,24,1,{'address': {'address_name': '충남 천안시 동남구 성황동 1...,태양빌라
688,4413110200100010001011579,2,4413110200,충청남도,천안시 동남구,성황동,NaN,0,1,1,0,{'address': {'address_name': '충남 천안시 동남구 성황동 1...,태양빌라
695,4413110200100010017010239,1,4413110200,충청남도,천안시 동남구,성황동,NaN,0,1,17,1,{'address': {'address_name': '충남 천안시 동남구 성황동 1...,장군빌라
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85717,4413325628101250009034225,3,4413325628,충청남도,천안시 서북구,직산읍,부송리,0,125,13,0,{'address': {'address_name': '충남 천안시 서북구 직산읍 부...,유원빌라트
85718,4413325628101250009034225,4,4413325628,충청남도,천안시 서북구,직산읍,부송리,0,125,14,0,{'address': {'address_name': '충남 천안시 서북구 직산읍 부...,유원빌라트
86381,4413325630101250009036633,1,4413325630,충청남도,천안시 서북구,직산읍,자은가리,0,125,9,1,{'address': {'address_name': '충남 천안시 서북구 직산읍 부...,유원빌라트
88318,4413331021100730006018496,2,4413331021,충청남도,천안시 서북구,입장면,하장리,0,73,44,0,{'address': {'address_name': '충남 천안시 서북구 입장면 하...,롯데빌라


In [9]:
apt_villa = pd.concat([apt,villa],axis=0)
print(len(apt)+len(villa)==len(apt_villa))

apt_villa.head(2)
apt_villa_code = list(apt_villa['법정동코드'].values)


True


,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부,byAPI,건물명
132,4413110100100430000014153,1,4413110100,충청남도,천안시 동남구,대흥동,NaN,0,43,0,1,{'address': {'address_name': '충남 천안시 동남구 대흥동 4...,천안맨션아파트
489,4413110100101650000019452,1,4413110100,충청남도,천안시 동남구,대흥동,NaN,0,165,0,1,{'address': {'address_name': '충남 천안시 동남구 대흥동 1...,미도아파트


In [15]:
apt_xl= pd.read_excel('/Users/iseunghyeon/Downloads/공동주택_단지목록_Code_20221129.xlsx')
apt_xl.head(2)
apt_xl.dropna(subset=['시군구'],inplace=True)
apt_xl = apt_xl[apt_xl['시군구'].str.contains('천안')]
apt_xl_code = list(apt_xl['법정동코드'].values)


,No,광역시도,시군구,동읍면,리,건물명,법정동코드,단지코드
0,1,강원도,강릉시,견소동,NaN,송정해변신도브래뉴,4215011500,A21015002
1,2,강원도,강릉시,견소동,NaN,한신휴플러스아파트,4215011500,A21015001


In [26]:
apt_xl_code_toApi = [elm for elm in apt_xl_code if elm not in apt_villa_code]
len(apt_xl_code_toApi),len(apt_xl_code)

(24, 316)

In [34]:
apt_xl_toApi =apt_xl[apt_xl['법정동코드'].isin(apt_xl_code_toApi)]
len(apt_xl_toApi)
apt_xl_toApi.head(2)

24

,No,광역시도,시군구,동읍면,리,건물명,법정동코드,단지코드
17877,17878,충청남도,천안동남구,목천읍,서리,재동보라,4413125021,A33074102
17883,17884,충청남도,천안동남구,목천읍,운전리,연합초원아파트,4413125029,A10025557


In [37]:
import requests
import xml.etree.ElementTree as ET
apiKey='2Z4X+NCMmHm8pFVALLw9qdxwZN4RZOilmbir7hj1MoaKCFCm0TPJG0WT7eCqNm3ijXOkg30FgQF7IRr5d8rUQg=='
url = 'http://apis.data.go.kr/1613000/AptBasisInfoService1/getAphusBassInfo'

apt_xl_toApi.reset_index(inplace=True,drop=True)

apt_xl_toApi['byApi2'] = None
for idx in range(len(apt_xl_toApi)):
    kaptCode = apt_xl_toApi.at[idx,'단지코드']

    params ={'serviceKey' : apiKey, 'kaptCode' : kaptCode } 

    response = requests.get(url, params=params)
    api = response.text
    root = ET.fromstring(api)
    data_dict = {}
    for item in root.findall(".//item/*"):
        data_dict[item.tag] = item.text

    apt_xl_toApi.at[idx,'byApi2'] = data_dict




In [ ]:
apt_xl_toApi

In [ ]:
apt_xl_toApi.at[0,'byApi2']['doroJuso']

apt_xl_toApi['주소'] = apt_xl_toApi['byApi2'].apply(lambda elm:elm.get('kaptAddr'))
apt_xl_toApi

In [58]:
apt_villa.head(2)

,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부,byAPI,건물명
132,4413110100100430000014153,1,4413110100,충청남도,천안시 동남구,대흥동,NaN,0,43,0,1,{'address': {'address_name': '충남 천안시 동남구 대흥동 4...,천안맨션아파트
489,4413110100101650000019452,1,4413110100,충청남도,천안시 동남구,대흥동,NaN,0,165,0,1,{'address': {'address_name': '충남 천안시 동남구 대흥동 1...,미도아파트


In [66]:
apt_xl_toApi.head(2)

,No,광역시도,시군구,동읍면,리,건물명,법정동코드,단지코드,byApi2,도로명주소,주소
0,17878,충청남도,천안동남구,목천읍,서리,재동보라,4413125021,A33074102,"{'bjdCode': '4413125021', 'codeAptNm': '아파트', ...",충청남도 천안동남구 목천읍 서리 58-1 재동보라,충청남도 천안동남구 목천읍 서리 58-1 재동보라
1,17884,충청남도,천안동남구,목천읍,운전리,연합초원아파트,4413125029,A10025557,"{'bjdCode': '4413125029', 'codeAptNm': '아파트', ...",충청남도 천안동남구 목천읍 운전리 380-3 연합초원아파트,충청남도 천안동남구 목천읍 운전리 380-3 연합초원아파트


In [ ]:
apt_xl_toApiByKakao = address2api(apt_xl_toApi,apiKey)

In [107]:
apt_xl_toApiByKakao.head(2)


,No,광역시도,시군구,동읍면,리,건물명,법정동코드,단지코드,byApi2,도로명주소,주소,byAPI
0,17878,충청남도,천안동남구,목천읍,서리,재동보라,4413125021,A33074102,"{'bjdCode': '4413125021', 'codeAptNm': '아파트', ...",충청남도 천안동남구 목천읍 서리 58-1 재동보라,충청남도 천안동남구 목천읍 서리 58-1 재동보라,{'address': {'address_name': '충남 천안시 동남구 목천읍 서...
1,17884,충청남도,천안동남구,목천읍,운전리,연합초원아파트,4413125029,A10025557,"{'bjdCode': '4413125029', 'codeAptNm': '아파트', ...",충청남도 천안동남구 목천읍 운전리 380-3 연합초원아파트,충청남도 천안동남구 목천읍 운전리 380-3 연합초원아파트,{'address': {'address_name': '충남 천안시 동남구 목천읍 운...


In [108]:
apt_xl_toApiByKakao_will_merge = apt_xl_toApiByKakao[['법정동코드','byAPI']]
apt_xl_toApiByKakao_will_merge.head(1)


,법정동코드,byAPI
0,4413125021,{'address': {'address_name': '충남 천안시 동남구 목천읍 서...


In [114]:
concat_df = pd.merge(apt_villa,apt_xl_toApiByKakao_will_merge,on=['법정동코드'],how='outer')
concat_df['byAPI'] = concat_df['byAPI_x'].fillna(concat_df['byAPI_y'])
concat_df.drop(['byAPI_x','byAPI_y'],axis=1,inplace=True)
concat_df

,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부,건물명,byAPI
0,4413110100100430000014153,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,43.0,0.0,1.0,천안맨션아파트,{'address': {'address_name': '충남 천안시 동남구 대흥동 4...
1,4413110100101650000019452,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,165.0,0.0,1.0,미도아파트,{'address': {'address_name': '충남 천안시 동남구 대흥동 1...
2,4413110100102140000000001,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,214.0,0.0,1.0,빌라,{'address': {'address_name': '충남 천안시 동남구 대흥동 2...
3,4413110100102370000022867,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,237.0,0.0,1.0,반석빌라,{'address': {'address_name': '충남 천안시 동남구 대흥동 2...
4,4413110200100340003010648,1.0,4413110200,충청남도,천안시 동남구,성황동,NaN,0.0,34.0,3.0,1.0,성황비둘기아파트,{'address': {'address_name': '충남 천안시 동남구 성황동 3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,NaN,NaN,4413325625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'address': {'address_name': '충남 천안시 서북구 직산읍 수...
1185,NaN,NaN,4413310600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'address': {'address_name': '충남 천안시 서북구 차암동 5...
1186,NaN,NaN,4413310600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'address': {'address_name': '충남 천안시 서북구 차암동 4...
1187,NaN,NaN,4413310600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'address': {'address_name': '충남 천안시 서북구 차암동 4...


In [115]:
len(apt_villa)+len(apt_xl_toApiByKakao_will_merge) == len(concat_df)

True

In [116]:
concat_df.to_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/주거단지/addressByAPIpickle0818.pickle')

In [117]:
concat_df

,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부,건물명,byAPI
0,4413110100100430000014153,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,43.0,0.0,1.0,천안맨션아파트,{'address': {'address_name': '충남 천안시 동남구 대흥동 4...
1,4413110100101650000019452,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,165.0,0.0,1.0,미도아파트,{'address': {'address_name': '충남 천안시 동남구 대흥동 1...
2,4413110100102140000000001,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,214.0,0.0,1.0,빌라,{'address': {'address_name': '충남 천안시 동남구 대흥동 2...
3,4413110100102370000022867,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,237.0,0.0,1.0,반석빌라,{'address': {'address_name': '충남 천안시 동남구 대흥동 2...
4,4413110200100340003010648,1.0,4413110200,충청남도,천안시 동남구,성황동,NaN,0.0,34.0,3.0,1.0,성황비둘기아파트,{'address': {'address_name': '충남 천안시 동남구 성황동 3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,NaN,NaN,4413325625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'address': {'address_name': '충남 천안시 서북구 직산읍 수...
1185,NaN,NaN,4413310600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'address': {'address_name': '충남 천안시 서북구 차암동 5...
1186,NaN,NaN,4413310600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'address': {'address_name': '충남 천안시 서북구 차암동 4...
1187,NaN,NaN,4413310600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'address': {'address_name': '충남 천안시 서북구 차암동 4...


In [135]:
공동주택 = pd.read_csv('/Users/iseunghyeon/Downloads/충청남도 천안시_공동주택 현황_20221230.csv',encoding='cp949')
공동주택
# 주소칼럼에서, 카카오 api가 읽을 수 없는 주소이름은, 네이버 지번 명을 통해 검색한다..
# 충청남도 천안시 서북구 월봉4로 140-14/16(쌍용동)	-> 충청남도 천안시 서북구 쌍용동 1538 [지번주소]
공동주택.loc[공동주택['주소']=='충청남도 천안시 서북구 월봉4로 140-14/16(쌍용동)','주소'] = '충청남도 천안시 서북구 쌍용동 1538'
공동주택.loc[공동주택['주소']=='충청남도 천안시 서북구 두정공단2길 26(두정동)','주소'] = '충남 천안시 서북구 두정공원2길 26 두정역범양레우스 알파'

,지역,공동주택명,주소,세대수,층수,동수,준공연도,관리실번호
0,천안시,한성스위트빌아파트(1차),충청남도 천안시 서북구 부성6길 11(두정동),600,15,6,2002-08-26,041-554-1133
1,천안시,계룡리슈빌아파트,충청남도 천안시 서북구 부성8길 29(두정동),386,15,4,2004-03-18,041-557-1472
2,천안시,두정경남아너스빌,충청남도 천안시 서북구 두정중11길 17(두정동),458,15,6,2004-06-10,041-556-3307
3,천안시,두정e편한세상(1차),충청남도 천안시 서북구 두정중2길 12(두정동),466,15,9,2004-06-25,041-561-8040
4,천안시,두정푸르지오아파트(3차),충청남도 천안시 서북구 성정두정로 142(두정동),326,15,4,2003-09-09,041-554-9116
...,...,...,...,...,...,...,...,...
408,천안시,천안역사동아라이크텐아파트,충청남도 천안시 서북구 천안천4길 20(와촌동),992,48,4,2021-04-30,041-579-9801
409,천안시,브리엔츠아파트,충청남도 천안시 서북구 삼은2길 26(직산읍),99,16,1,2019-09-04,NaN
410,천안시,두정한화포레나,충청남도 천안시 서북구 천안대로 980-17(두정동),1067,29,13,2022-03-30,041-552-9300
411,천안시,천안필하우스에듀시티1단지,충청남도 천안시 동남구 성황로 40(문화동),1532,28,13,2022-04-19,041-622-8574


In [136]:
공동주택byApi = address2api(공동주택,apiKey)
공동주택byApi

100%|██████████| 413/413 [00:40<00:00, 10.30it/s]


,지역,공동주택명,주소,세대수,층수,동수,준공연도,관리실번호,byAPI
0,천안시,한성스위트빌아파트(1차),충청남도 천안시 서북구 부성6길 11(두정동),600,15,6,2002-08-26,041-554-1133,{'address': {'address_name': '충남 천안시 서북구 두정동 1...
1,천안시,계룡리슈빌아파트,충청남도 천안시 서북구 부성8길 29(두정동),386,15,4,2004-03-18,041-557-1472,{'address': {'address_name': '충남 천안시 서북구 두정동 1...
2,천안시,두정경남아너스빌,충청남도 천안시 서북구 두정중11길 17(두정동),458,15,6,2004-06-10,041-556-3307,{'address': {'address_name': '충남 천안시 서북구 두정동 1...
3,천안시,두정e편한세상(1차),충청남도 천안시 서북구 두정중2길 12(두정동),466,15,9,2004-06-25,041-561-8040,{'address': {'address_name': '충남 천안시 서북구 두정동 1...
4,천안시,두정푸르지오아파트(3차),충청남도 천안시 서북구 성정두정로 142(두정동),326,15,4,2003-09-09,041-554-9116,{'address': {'address_name': '충남 천안시 서북구 두정동 1...
...,...,...,...,...,...,...,...,...,...
408,천안시,천안역사동아라이크텐아파트,충청남도 천안시 서북구 천안천4길 20(와촌동),992,48,4,2021-04-30,041-579-9801,{'address': {'address_name': '충남 천안시 서북구 와촌동 1...
409,천안시,브리엔츠아파트,충청남도 천안시 서북구 삼은2길 26(직산읍),99,16,1,2019-09-04,NaN,{'address': {'address_name': '충남 천안시 서북구 직산읍 삼...
410,천안시,두정한화포레나,충청남도 천안시 서북구 천안대로 980-17(두정동),1067,29,13,2022-03-30,041-552-9300,{'address': {'address_name': '충남 천안시 서북구 두정동 2...
411,천안시,천안필하우스에듀시티1단지,충청남도 천안시 동남구 성황로 40(문화동),1532,28,13,2022-04-19,041-622-8574,{'address': {'address_name': '충남 천안시 동남구 문화동 1...


In [137]:
공동주택byApi[공동주택byApi['byAPI'].isna()]


,지역,공동주택명,주소,세대수,층수,동수,준공연도,관리실번호,byAPI


In [151]:
concat_df.loc[0,'byAPI'].keys()

concat_df.loc[101,'byAPI'].get('road_address').get('address_name')
concat_df.loc[10,'byAPI'].get('road_address')


dict_keys(['address', 'address_name', 'address_type', 'road_address', 'x', 'y'])

'충남 천안시 동남구 원성16길 9-1'

{'address_name': '충남 천안시 동남구 성황6길 16',
 'building_name': '성광빌라',
 'main_building_no': '16',
 'region_1depth_name': '충남',
 'region_2depth_name': '천안시 동남구',
 'region_3depth_name': '성황동',
 'road_name': '성황6길',
 'sub_building_no': '',
 'underground_yn': 'N',
 'x': '127.149815523737',
 'y': '36.8158187034778',
 'zone_no': '31121'}

In [152]:
m1 = concat_df.copy()
m2 = 공동주택byApi.copy()


<!-- /Users/iseunghyeon/Desktop/프로젝트/data/제조업체/cmp2F(수정).csv -->


In [153]:
def get_address_name(row):
    if row['byAPI'] is None:
        return None
    return row['byAPI'].get('road_address',{}).get('address_name',None)

m1['address_name'] = m1.apply(get_address_name,axis=1)
m2['address_name'] = m1.apply(get_address_name,axis=1)


In [ ]:
m1[m1['address_name']=='충남 천안시 동남구 성불사길 17']

In [161]:
m1.head(2)
m2.head(2)


,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부,건물명,byAPI,address_name
0,4413110100100430000014153,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,43.0,0.0,1.0,천안맨션아파트,{'address': {'address_name': '충남 천안시 동남구 대흥동 4...,충남 천안시 동남구 대흥로 267
1,4413110100101650000019452,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,165.0,0.0,1.0,미도아파트,{'address': {'address_name': '충남 천안시 동남구 대흥동 1...,충남 천안시 동남구 옛시청길 6


,지역,공동주택명,주소,세대수,층수,동수,준공연도,관리실번호,byAPI,address_name
0,천안시,한성스위트빌아파트(1차),충청남도 천안시 서북구 부성6길 11(두정동),600,15,6,2002-08-26,041-554-1133,{'address': {'address_name': '충남 천안시 서북구 두정동 1...,충남 천안시 동남구 대흥로 267
1,천안시,계룡리슈빌아파트,충청남도 천안시 서북구 부성8길 29(두정동),386,15,4,2004-03-18,041-557-1472,{'address': {'address_name': '충남 천안시 서북구 두정동 1...,충남 천안시 동남구 옛시청길 6


In [162]:
m2_2 = m2[['byAPI','address_name']]
m2_2.head(2)

,byAPI,address_name
0,{'address': {'address_name': '충남 천안시 서북구 두정동 1...,충남 천안시 동남구 대흥로 267
1,{'address': {'address_name': '충남 천안시 서북구 두정동 1...,충남 천안시 동남구 옛시청길 6


In [158]:
m1_address_name_list = list(m1['address_name'].values)
m2_address_name_list = list(m2['address_name'].values)
len(m1_address_name_list),len(m2_address_name_list)

(1189, 413)

In [159]:
set_m1_address_name_list = set(m1_address_name_list)
set_m2_address_name_list = set(m2_address_name_list)
inter_list = set_m1_address_name_list.intersection(set_m2_address_name_list)
len(inter_list)

297

In [167]:
# outer 조인을 통하여, 충청남도 천안시_공동주택 현황_20221230.csv 파일과의 데이터와 결합을 한다.
m_merge = pd.merge(m1,m2_2,on=['address_name'],how='outer')
m_merge.head(2)
m_merge['byAPI'] = m_merge['byAPI_x'].fillna(m_merge['byAPI_y'])
m_merge.drop(['byAPI_x','byAPI_y'],axis=1,inplace=True)
m_merge.head(2)
len(m_merge)

,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부,건물명,byAPI_x,address_name,byAPI_y
0,4413110100100430000014153,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,43.0,0.0,1.0,천안맨션아파트,{'address': {'address_name': '충남 천안시 동남구 대흥동 4...,충남 천안시 동남구 대흥로 267,{'address': {'address_name': '충남 천안시 서북구 두정동 1...
1,4413110100101650000019452,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,165.0,0.0,1.0,미도아파트,{'address': {'address_name': '충남 천안시 동남구 대흥동 1...,충남 천안시 동남구 옛시청길 6,{'address': {'address_name': '충남 천안시 서북구 두정동 1...


,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부,건물명,address_name,byAPI
0,4413110100100430000014153,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,43.0,0.0,1.0,천안맨션아파트,충남 천안시 동남구 대흥로 267,{'address': {'address_name': '충남 천안시 동남구 대흥동 4...
1,4413110100101650000019452,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,165.0,0.0,1.0,미도아파트,충남 천안시 동남구 옛시청길 6,{'address': {'address_name': '충남 천안시 동남구 대흥동 1...


1669

In [174]:
m_merge_uq = m_merge.drop_duplicates(subset='address_name',keep='first')
len(m_merge_uq)
m_merge_uq.head(2)

718

,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부,건물명,address_name,byAPI
0,4413110100100430000014153,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,43.0,0.0,1.0,천안맨션아파트,충남 천안시 동남구 대흥로 267,{'address': {'address_name': '충남 천안시 동남구 대흥동 4...
1,4413110100101650000019452,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,165.0,0.0,1.0,미도아파트,충남 천안시 동남구 옛시청길 6,{'address': {'address_name': '충남 천안시 동남구 대흥동 1...


In [176]:
m_merge_uq.to_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/주거단지/addressByAPIpickle0819.pickle')

In [181]:
m_merge_uq.loc[1645,'byAPI'].get('road_address').get('building_name')

'보라아파트'

In [182]:
# 건물명 채우기
def fill_building_name(row):
    if pd.isna(row['건물명']):
        return row['byAPI'].get('road_address',{}).get('building_name',None)
    return row['건물명']
m_merge_uq['건물명'] = m_merge_uq.apply(fill_building_name,axis=1)


In [185]:
m_merge_uq.to_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/주거단지/addressByAPIpickle08191010.pickle')

In [55]:

# apt_xl_toApi[apt_xl_toApi['단지코드']=='A10023818']['byApi2'].values
apt_xl_toApi[apt_xl_toApi['단지코드']=='A10023818']['byApi2'].get('kaptAddr')

In [ ]:
import requests
import xml.etree.ElementTree as ET
# apiKey='2Z4X%2BNCMmHm8pFVALLw9qdxwZN4RZOilmbir7hj1MoaKCFCm0TPJG0WT7eCqNm3ijXOkg30FgQF7IRr5d8rUQg%3D%3D'
apiKey='2Z4X+NCMmHm8pFVALLw9qdxwZN4RZOilmbir7hj1MoaKCFCm0TPJG0WT7eCqNm3ijXOkg30FgQF7IRr5d8rUQg=='
url = 'http://apis.data.go.kr/1613000/AptBasisInfoService1/getAphusBassInfo'

params ={'serviceKey' : apiKey, 'kaptCode' : 'A33109016' }

response = requests.get(url, params=params)
# print(response.content)
# print(response.text)
api = response.text
root = ET.fromstring(api)
data_dict = {}
for item in root.findall(".//item/*"):
    data_dict[item.tag] = item.text

data_dict


In [24]:
apt_villa_cp = apt_villa.copy()
apt_xl_cp = apt_xl.copy()
# apt_xl_cp = apt_xl_cp[['건물명','법정동코드','단지코드']]
# apt_xl_cp.head(2)

,건물명,법정동코드,단지코드
17863,구성 현대I-PARK 아파트,4413110800,A33006001
17864,구성동 쌍용아파트,4413110800,A33006002


In [17]:
temp = apt_xl_code - apt_villa_code
len(temp)

TypeError: unsupported operand type(s) for -: 'list' and 'list'